In [ ]:
#Install Packages
!pip install faiss-cpu
!pip install sentence-transformers

In [3]:
# import necessary libraries
import pandas as pd
pd.set_option('display.max_colwidth', 100)

In [4]:
df = pd.read_csv("sample_text.csv")
df.shape

(8, 2)

In [6]:
df

,text,category
0,Meditation and yoga can improve mental health,Health
1,"Fruits, whole grains and vegetables helps control blood pressure",Health
2,These are the latest fashion trends for this week,Fashion
3,Vibrant color jeans for male are becoming a trend,Fashion
4,The concert starts at 7 PM tonight,Event
5,Navaratri dandiya program at Expo center in Mumbai this october,Event
6,Exciting vacation destinations for your next trip,Travel
7,Maldives and Srilanka are gaining popularity in terms of low budget vacation places,Travel


In [ ]:
df.text[0]

'Meditation and yoga can improve mental health'

### Step 1 : Create source embeddings for the text column

In [37]:
from sentence_transformers import SentenceTransformer

In [38]:
encoder = SentenceTransformer("all-mpnet-base-v2")
vectors = encoder.encode(df.text)

In [39]:
vectors.shape

(8, 768)

In [40]:
vectors

array([[-0.00247397,  0.03626722, -0.05290459, ..., -0.09152357,
        -0.03969998, -0.04330488],
       [-0.03357267,  0.00980518, -0.03250128, ..., -0.05165467,
         0.02245889, -0.03156183],
       [-0.01865323, -0.04051315, -0.01235388, ...,  0.00610584,
        -0.07179647,  0.02773854],
       ...,
       [-0.0006646 ,  0.04252123, -0.05645508, ...,  0.01315468,
        -0.03183568, -0.04357662],
       [-0.03317154,  0.03252456, -0.0248484 , ...,  0.01174423,
         0.05747125,  0.00571021],
       [-0.00166395,  0.00413826, -0.04597082, ...,  0.02008526,
         0.05656243, -0.00161596]], dtype=float32)

In [41]:
dim = vectors.shape[1]
dim

768

### Step 2 : Build a FAISS Index for vectors

In [42]:
import faiss

index = faiss.IndexFlatL2(dim)
index #768

<faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x00000236312EFBD0> >

### Step 3 : Normalize the source vectors (as we are using L2 distance to measure similarity) and add to the index

In [43]:
index.add(vectors)

In [44]:
index

<faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x00000236312EFBD0> >

### Step 4 : Encode search text using same encorder and normalize the output vector

In [ ]:
search_query = "I want to buy a polo t-shirt" # !!!sementic not keyword
# search_query = "looking for places to visit during the holidays"
# search_query = "An apple a day keeps the doctor away"
vec = encoder.encode(search_query)
vec.shape

(768,)

In [46]:
vec

array([ 1.03882868e-02,  2.78686509e-02, -1.18618757e-02,  1.81327220e-02,
        1.21983094e-03, -1.42995883e-02,  1.96229331e-02,  2.07198113e-02,
       -2.23660879e-02,  4.75626625e-02,  1.77976023e-02, -8.00332334e-03,
        2.53419559e-02,  5.26149124e-02,  8.44941940e-03, -1.63944028e-02,
        1.02661615e-02, -2.74856314e-02,  8.37067366e-02, -1.52885308e-02,
        1.67682841e-02, -3.97141138e-03, -2.74348482e-02,  5.02091721e-02,
       -8.36522318e-03, -4.74075824e-02,  2.36915536e-02, -1.01186372e-02,
       -2.82840487e-02,  7.94263929e-03,  4.21436056e-02, -4.19374602e-03,
       -1.91232134e-02, -3.12785506e-02,  1.24685050e-06, -1.04428418e-02,
       -2.19971314e-02, -8.66928175e-02, -1.88496290e-03, -2.54772156e-02,
       -9.72973090e-03,  7.93310478e-02, -3.55963483e-02, -3.05374182e-04,
       -1.12392167e-02, -3.88932303e-02,  5.49313761e-02,  1.35207996e-01,
       -8.19147378e-02,  1.18785119e-02, -9.01845470e-03,  1.92544367e-02,
       -2.82723475e-02, -

In [47]:
# need 2 dims
import numpy as np
svec = np.array(vec).reshape(1,-1)
svec.shape

(1, 768)

In [48]:
svec

array([[ 1.03882868e-02,  2.78686509e-02, -1.18618757e-02,
         1.81327220e-02,  1.21983094e-03, -1.42995883e-02,
         1.96229331e-02,  2.07198113e-02, -2.23660879e-02,
         4.75626625e-02,  1.77976023e-02, -8.00332334e-03,
         2.53419559e-02,  5.26149124e-02,  8.44941940e-03,
        -1.63944028e-02,  1.02661615e-02, -2.74856314e-02,
         8.37067366e-02, -1.52885308e-02,  1.67682841e-02,
        -3.97141138e-03, -2.74348482e-02,  5.02091721e-02,
        -8.36522318e-03, -4.74075824e-02,  2.36915536e-02,
        -1.01186372e-02, -2.82840487e-02,  7.94263929e-03,
         4.21436056e-02, -4.19374602e-03, -1.91232134e-02,
        -3.12785506e-02,  1.24685050e-06, -1.04428418e-02,
        -2.19971314e-02, -8.66928175e-02, -1.88496290e-03,
        -2.54772156e-02, -9.72973090e-03,  7.93310478e-02,
        -3.55963483e-02, -3.05374182e-04, -1.12392167e-02,
        -3.88932303e-02,  5.49313761e-02,  1.35207996e-01,
        -8.19147378e-02,  1.18785119e-02, -9.01845470e-0

In [49]:
# faiss.normalize_L2(svec)

### Step 5: Search for similar vector in the FAISS index created

In [58]:
index.search(svec, k=2)

(array([[1.3844836, 1.4039094]], dtype=float32), array([[3, 2]], dtype=int64))

In [62]:
distances, I = index.search(svec, k=2)
distances

array([[1.3844836, 1.4039094]], dtype=float32)

In [63]:
I

array([[3, 2]], dtype=int64)

In [66]:
df.loc[[3,2]]

,text,category
3,Vibrant color jeans for male are becoming a trend,Fashion
2,These are the latest fashion trends for this week,Fashion


In [64]:
I.tolist()

[[3, 2]]

In [65]:
row_indices = I.tolist()[0]
row_indices

[3, 2]

In [56]:
df.loc[row_indices]

,text,category
3,Vibrant color jeans for male are becoming a trend,Fashion
2,These are the latest fashion trends for this week,Fashion


In [57]:
search_query

'I want to buy a polo t-shirt'

You can see that the two results from the dataframe are similar to a search_query